In [1]:
import numpy as np

In [5]:
np.load(r"D:\Project\Datasets\randomSampling_bone_controller\npy2\dataBoneSet103.npy").shape

(1000, 4473)

## Test

In [ ]:
from paddle.vision import transforms as T
import numpy as np
from dataset import JointCtrl
import os
value_0 = np.load("./value_0.npy")
mean = np.array([0.00174034, -0.0296329, 0.08699034, 0.4884157, 0.13937208, 0.1928098], dtype=np.float64)
std = np.array([0.47395173, 0.5340112, 0.62341311, 17.59106008, 7.27811802, 9.06478785], dtype=np.float64)
def get_train_transforms():
    return T.Compose([
        # T.ToTensor(),
        T.Normalize(mean=mean, std=std),])

In [ ]:
data_root_path = r'D:\Project\Datasets\randomSampling'
data_len = 10
joint_path_list = []
ctrl_path_list = []
transform = get_train_transforms()
for i in range(data_len):
    joint_path_list.append(os.path.join(data_root_path, f"dataBoneSet{i}.npy"))
    ctrl_path_list.append(os.path.join(data_root_path, f"dataCtrlSet_{i}.npy"))

# data = JointCtrl(joint_path_list, ctrl_path_list)
dataset_train = JointCtrl(joint_path_list[:int(0.8*data_len)], 
                            ctrl_path_list[:int(0.8*data_len)], 
                            transform=transform,
                            value_0=value_0)
dataset_val = JointCtrl(joint_path_list[int(0.8*data_len):], 
                            ctrl_path_list[int(0.8*data_len):], 
                            transform=transform,
                            value_0=value_0)


In [ ]:
joint_100 = (np.load(joint_path_list[0])-value_0)[100]

In [ ]:
(joint_100[600:606]-mean)/std

In [ ]:
dataset_train.__getitem__(100)[0][:,0,100]

In [ ]:
data_list = (data_list[3] - mean.reshape((6,1,1))) / std.reshape((6,1,1))

In [ ]:
data_list[:,0,0]

In [ ]:
data_list = []
label_list = []
for joint_path, ctrl_path in zip(joint_path_list, ctrl_path_list):
    # d1 = np.load(joint_path).astype(dtype=np.float32)
    # print(d1[0][-16:])
    d = np.load(joint_path) - value_0
    data_list.extend(d.reshape(1000, 1, -1, 6)
                        .transpose(0, 3, 1, 2))
    label_list.extend(np.load(ctrl_path))

In [ ]:
np.set_printoptions(threshold=np.inf)

In [ ]:
data_list[0] =(data_list[0] - mean.reshape((6,1,1))) / std.reshape((6,1,1)) 

In [ ]:
data_ = np.array(data_list)
print(data_.shape)

In [ ]:
# 对data随机取1000个样本，计算均值和方差
data = data_
# data = data_[np.random.choice(data_.shape[0], 50000, replace=False), :, :, :]
# 计算data第2维的均值和方差
mean = np.mean(data, axis=(0, 2, 3), keepdims=False)
std = np.std(data, axis=(0, 2, 3), keepdims=False)
print(mean, std)

In [ ]:
mean

In [ ]:
dataset_train.__len__()

In [ ]:
data = np.array(data_list)
j = 30
data[1000,:,0,1]==dataset_train.__getitem__(2)[0][:,0,j]

In [ ]:
d1 = np.load(joint_path_list[0]).astype(dtype=np.float32)
d2 = d1.reshape(1000, 1, -1, 6)
d3 = d2.transpose(0, 3, 1, 2)
d3[0, :, 0, 1]

In [ ]:
data.shape

In [ ]:
# 将data第二维转为(6,1,-1)
data = data.reshape(data.shape[0], 1, -1, 6).transpose(0,3,1,2)
data[0,:,0,0]

## Model Save

In [ ]:
import numpy as np
value_0 = np.load("./value_0.npy")
joint = np.loadtxt('./Pia_jointAnimation.txt')-value_0
ctrl = np.loadtxt('./Pia_ctrlAnimation.txt')

In [ ]:
from model import build_model
import paddle
model = build_model(model_name='resnet18', in_features=6, out_features=220)
model_state = paddle.load("./output/epoch_1000/model.pdparams")
model.set_dict(model_state)
model_jit = paddle.jit.to_static(model)
paddle.jit.save(model_jit, "./inference/model", input_spec=[paddle.static.InputSpec(shape=[None, 6, 887, 6], name="x",dtype='float32')])

In [ ]:
import paddle
paddle.enable_static()
exe = paddle.static.Executor(paddle.CPUPlace())
[inference_program, feed_target_names, fetch_targets] = (paddle.static.load_inference_model('./inference/model', exe))

In [ ]:
from paddle.vision import transforms as T

def get_train_transforms():
    return T.Compose([
        # T.ToTensor(),
        T.Normalize(mean=[0.00174034, -0.0296329, 0.08699034, 0.4884157, 0.13937208, 0.1928098], 
                    std=[0.47395173, 0.5340112, 0.62341311, 17.59106008, 7.27811802, 9.06478785])])
transform = get_train_transforms()

In [ ]:
joint[0].shape

In [ ]:
jiont_test = joint[0].reshape(-1,6)
jiont_test_ = jiont_test[np.newaxis, :]

# 将joint_test_第三维标准化
jiont_test__ = transform(jiont_test_)
jiont_test__ = jiont_test__.reshape(1,6,887,6).astype(np.float32)
# jiont_test__ = np.array([jiont_test__], dtype=np.float32)
# jiont_test__ = paddle.to_tensor(jiont_test__, dtype='float32')


In [ ]:
results = exe.run(inference_program,
                feed={feed_target_names[0]: jiont_test__},
                fetch_list=fetch_targets)

In [ ]:
results[0]